# Key features of langGraph

1. Looping and branching capabilities

Supports conditional statements and loop structures, allowing dynamic excution based on route

2. State Persistance:
Automatically preserves state supporting pause and resume for long running conversations

3. Human Machine interaction support

Allows inserting of human review during execution, supporting state editing and modification with flexible interaction control mechanism

4. Streaming processing
supports streaming output and real-time feedback on execution status to enhance user experience

5. Seemless Integration with langchain


# Core Components of LangGraph

1. Nodes
2. edges
3. conditional edges
4. state

# Reflection Agent Pattern

N.B will always have something that generates something and another that critics this 

A reflection agent pattern is an AI system pattern that can look at its own outputs and think about them/ make them better
A basic reflection agent system typically consist of:
1. A generator agent
2. A reflector agent

# Types of Reflection Agents
1. Basic Reflection agents
2. Reflexion agents
3. Language Agents Tree Search(LATS)

In [19]:
# Basic Reflection agent

# CHAINS 

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

generation_prompt = ChatPromptTemplate.from_messages([
("system", "You are a tweeter techie tasked with generating a tweet about the user's input. "
            "Generate the best tweet possible. "
            "If the user criticizes your tweet, reflect on it and improve it."),
MessagesPlaceholder(variable_name="messages"),
])


reflect_prompt = ChatPromptTemplate.from_messages([
("system", "You are a viral Twitter influencer grading a tweet. "
            "Generate a critique and recommendation for the user's tweet. "
            "Always provide a critique and recommendation. "
            "If the user criticizes your critique, reflect on it and improve it."),
MessagesPlaceholder(variable_name="messages"),
])
# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-pro",
#     temperature=0.2,
#     max_output_tokens=100,
# )
llm = ChatOpenAI(model="gpt-4o")
generation_chain = generation_prompt | llm
reflect_chain = reflect_prompt | llm





A MessageGraph: is a class provided by langGraph that we can use to orchestrate the flow of messages between different nodes. 
use cases include simplechatbox, routing decisions etc
If your app requires complex state manages consider using the `stateGraph`
Each node recieves the entire list of messages and can append to it or modify

In [38]:
from langsmith import utils
from dotenv import load_dotenv
import os

utils.get_env_var.cache_clear()

print("Loading environment variables...")
load_dotenv()
print(os.environ.get("LANGSMITH_API_KEY"))
print("Checking if tracing is enabled...")
# print(os.environ)

utils.tracing_is_enabled()

Loading environment variables...
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
Checking if tracing is enabled...


True

In [39]:
from typing import List, Sequence
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import END, MessageGraph
load_dotenv()

REFLECT = "reflect"
GENERATE = "generate"
graph = MessageGraph()


def generate_node(state):
    return generation_chain.invoke({
        "messages": state}
    )

def reflect_node(state):
    response =reflect_chain.invoke({
        "messages": state}
    )
    return[ HumanMessage(
        content=response.content
    )]

graph.add_node(
    GENERATE,
    generate_node,
    # input_variables=["messages"],
    # output_variables=["messages"],
)
graph.add_node(
    REFLECT,
    reflect_node,
    # input_variables=["messages"],
    # output_variables=["messages"],
)

graph.set_entry_point(GENERATE)

def should_continue(state: Sequence[BaseMessage]):
    """
    Check if the last message is a critique.
    """
    # if not state:
    #     return True
    # last_message = state[-1]
    # return isinstance(last_message, AIMessage) and "critique" not in last_message.content.lower()
    if len(state) > 2:
        return END
    return REFLECT


graph.add_conditional_edges(
    GENERATE,
    should_continue,
    # {
    #     True: REFLECT,
    #     False: END,
    # },
)

graph.add_edge(REFLECT, GENERATE)
app = graph.compile()
print(app.get_graph().draw_mermaid())
app.get_graph().print_ascii()

response =app.invoke(HumanMessage(
    content="I love the new features in the latest iPhone update! Can't wait to try them out.",
))

print("Response:", response)





---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__(<p>__start__</p>)
	generate(generate)
	reflect(reflect)
	__end__(<p>__end__</p>)
	__start__ --> generate;
	generate --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
+----------+   
| generate |   
+----------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   
Response: [HumanMessage(content="I love the new features in the latest iPhone update! Can't wait to try them out.", additional_kwargs={}, response_metadata={}, id='d730f646-f8ae-4081-b645-fb5ca13da243'), AIMessage(content="Unboxing the future! 📱✨ Just updated my iPhone and the new features are blowing my mind. Can't wait to dive in and explore all the possibilities! Who else is excited? #iPhoneUpdate #TechLovers #AppleMagic 🍏📲", additional_kwargs={'ref